In [12]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import math
import itertools

## Multivariable

In [ ]:
# Generate 100 2d random points
np.random.seed(0)

N = 100  # number of data samples
D = 2   # dimensions of data
C = math.comb(N, D-1)  # number of combinations of N choose D-1
X = np.random.multivariate_normal(mean=np.zeros(D), cov=np.eye(D), size=N)

In [ ]:
# find K matrix from the given data, see Theorem F.2
def find_K(X: np.ndarray) -> np.ndarray:
    K = np.zeros((N, C))
    row_idx = np.arange(N)

    # Generate possible combinations of d-1 elements from n elements
    idx_combinations = list(itertools.combinations(row_idx, D-1))

    for i in range(N):
        x_i = X[i]
        for j in range(C):
            x_j = X[idx_combinations[j]]
            
            # Compute outer product of every row in x_j and take L1 norm
            denominator = np.linalg.norm(x_j, ord=1)

            numerator = np.linalg.det(np.outer(x_i, x_j))
            numerator = np.maximum(numerator, 0)

            K[i, j] = numerator / denominator

    return K

K = find_K(X)
print(K.shape)

(100, 100)


In [ ]:
# Solve Eq 33
# L is label matrix
epsilon = 0.1
L = np.random.multivariate_normal(mean=np.zeros(D), cov=epsilon*np.eye(D), size=N)
L = L / epsilon

reg_lambda = 1
Z = cp.Variable((C, N))

# Define the optimization problem
regularization_term = reg_lambda * cp.sum([cp.norm(Z[:, i], 2) for i in range(N)])
objective = cp.Minimize(cp.norm(K.T @ Z - L, 'fro')**2 + regularization_term)

# Solve the optimization problem
problem = cp.Problem(objective)
problem.solve()

# Extract the optimal value and optimal Z
optimal_value = problem.value
optimal_Z = Z.value

In [ ]:
# Achieve NN
# They fucked up the notation in Eq 34..., 
# We should look Eq 48 of arxiv.org/pdf/2309.16512 
# I'm interpreting Z_j as the j-th row of Z, and x_i as x, which is function input
def NN(x: np.ndarray) -> np.ndarray:
    output = np.zeros(D)
    idx_combinations = list(itertools.combinations(np.arange(N), D-1))
    for j in range(C):
        Z_j = optimal_Z[j]
        x_j = X[idx_combinations[j]]

        numerator = np.linalg.det(np.outer(x, x_j))
        numerator = np.maximum(numerator, 0)

        denominator = np.linalg.norm(x_j, ord=1)

        output += Z_j * (numerator / denominator)

    return output
